In [ ]:
import pandas as pd
import re

# Load all sheets from the Excel file
file_path = '/Users/myself/Desktop/Walmart USA Serching for Growth/Distribution Centers.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)

def standardize_columns(data):
    # Standardize column names
    standard_names = {
        'DC': 'DC Number',
        'Year': 'Year Opened',
        'Sq Ft': 'Square Feet',
        # Add other variations as needed
    }
    return data.rename(columns={old: standard_names.get(old, old) for old in data.columns})

def clean_square_feet(value):
    # Remove commas and spaces, correct decimal points issues
    value = str(value).replace(',', '').replace(' ', '')
    parts = value.split('.')
    if len(parts) > 2:
        value = ''.join(parts[:-1]) + '.' + parts[-1]
    try:
        return float(value)
    except ValueError:
        return 0.0  # Default to 0.0 for any parsing errors

def extract_year(date_str):
    # Extract year from various formats
    if pd.isna(date_str):
        return None
    match = re.search(r'\b(19|20)\d{2}\b', str(date_str))
    return int(match.group(0)) if match else None

def extract_type(description):
    description = str(description)  # Convert to string to handle NaN values safely
    if "Crossdock" in description:
        return "Crossdock"
    elif "Import" in description:
        return "Import"
    elif "Distribution Center" in description:
        return "Distribution Center"
    return "Other"


def clean_sheet(data):
    # Clean individual sheets and extract necessary information
    data = standardize_columns(data)
    data['Square Feet'] = data['Square Feet'].apply(clean_square_feet)
    data['Year Opened'] = data['Year Opened'].apply(extract_year)
    data['Type'] = data['Description of Operation'].apply(extract_type)
    return data[['State', 'DC Number', 'Location', 'Square Feet', 'Year Opened', 'Type']]



In [ ]:

# Process and aggregate all sheets again with the updated extract_type function
cleaned_data = pd.concat([clean_sheet(sheet) for sheet in all_sheets.values() if not sheet.empty])
summary_stats = cleaned_data.groupby(['Year Opened', 'Type']).agg({'Square Feet': 'sum', 'DC Number': 'count'}).reset_index()
summary_stats.rename(columns={'DC Number': 'Count'}, inplace=True)
summary_stats.sort_values(by='Year Opened', inplace=True)

summary_stats


In [ ]:
# Function to validate and count all processed and filtered entries
def validate_and_count_entries(all_sheets):
    total_rows = 0
    processed_rows = 0

    for sheet_name, data in all_sheets.items():
        # Count total rows before any processing
        total_rows += len(data)

        # Apply cleaning functions
        data_cleaned = clean_sheet(data)
        
        # Count rows after processing (which have valid 'Year Opened' and type isn't 'Other' unless explicitly set)
        processed_rows += len(data_cleaned[data_cleaned['Year Opened'].notna() & (data_cleaned['Type'] != 'Other')])

    return total_rows, processed_rows

# Calculate total and processed entries
total_entries, processed_entries = validate_and_count_entries(all_sheets)

total_entries, processed_entries


In [ ]:
import re

def robust_extract_year(date_str):
    if pd.isna(date_str):
        return None
    # Check for simple year format first
    year_match = re.search(r'\b(19|20)\d{2}\b', str(date_str))
    if year_match:
        return int(year_match.group(0))
    
    # Check for year with quarters or months
    complex_match = re.search(r'(Q[1-4]|January|February|March|April|May|June|July|August|September|October|November|December)[\s,]*([12][90]\d{2})', str(date_str), re.IGNORECASE)
    if complex_match:
        return int(complex_match.group(2))
    
    # Check for any other numerical data that might represent a year
    fallback_match = re.search(r'([12][90]\d{2})', str(date_str))
    if fallback_match:
        return int(fallback_match.group(1))
    
    return None  # Return None if no valid year found

# Test the function with a variety of date formats
test_dates = ["Opened", "October, 2019", "Q4, 2018", "May, 2019", "135,287 | April, 2019", "2027", "NaN"]
extracted_years = [robust_extract_year(date) for date in test_dates]
extracted_years


In [ ]:
# Update the clean_sheet function to use the new robust_extract_year function for 'Year Opened'
def clean_sheet_updated(data, sheet_name):
    # Clean individual sheets and extract necessary information
    data = standardize_columns(data)
    data['Square Feet'] = data['Square Feet'].apply(clean_square_feet)
    data['Year Opened'] = data['Year Opened'].apply(robust_extract_year)
    data['Type'] = data['Description of Operation'].apply(lambda x: extract_type(x, sheet_name))
    return data[['State', 'DC Number', 'Location', 'Square Feet', 'Year Opened', 'Type']]

# Function to validate and count all processed and filtered entries using the updated clean_sheet function
def validate_and_count_entries_updated(all_sheets):
    total_rows = 0
    processed_rows = 0

    for sheet_name, data in all_sheets.items():
        if data.empty:
            continue
        total_rows += len(data)
        data_cleaned = clean_sheet_updated(data, sheet_name)
        processed_rows += len(data_cleaned[data_cleaned['Year Opened'].notna() & (data_cleaned['Type'] != 'Other')])

    return total_rows, processed_rows

# Recalculate total and processed entries with the updated date extraction logic
validate_and_count_entries_updated(all_sheets)


In [ ]:
import pandas as pd
import re

# Load all sheets from the Excel file
# file_path = 'path_to_your_file/Distribution Centers.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)

def standardize_columns(data):
    # Standardize column names
    standard_names = {
        'DC': 'DC Number',
        'Year': 'Year Opened',
        'Sq Ft': 'Square Feet',
    }
    return data.rename(columns={old: standard_names.get(old, old) for old in data.columns})

def clean_square_feet(value):
    # Remove commas and spaces, correct decimal points issues
    value = str(value).replace(',', '').replace(' ', '')
    parts = value.split('.')
    if len(parts) > 2:
        value = ''.join(parts[:-1]) + '.' + parts[-1]
    try:
        return float(value)
    except ValueError:
        return 0.0  # Default to 0.0 for any parsing errors

def robust_extract_year(date_str):
    if pd.isna(date_str):
        return None
    # Check for simple year format first
    year_match = re.search(r'\b(19|20)\d{2}\b', str(date_str))
    if year_match:
        return int(year_match.group(0))
    # Check for year with quarters or months
    complex_match = re.search(r'(Q[1-4]|January|February|March|April|May|June|July|August|September|October|November|December)[\s,]*([12][90]\d{2})', str(date_str), re.IGNORECASE)
    if complex_match:
        return int(complex_match.group(2))
    # Check for any other numerical data that might represent a year
    fallback_match = re.search(r'([12][90]\d{2})', str(date_str))
    if fallback_match:
        return int(fallback_match.group(1))
    return None

def extract_type(description, sheet_name):
    description = str(description)  # Convert to string to handle NaN values safely
    if "Crossdock" in description:
        return "Crossdock"
    elif "Import" in description:
        return "Import"
    elif "Distribution Center" in description:
        return "Distribution Center"
    return sheet_name  # Use the sheet name as the type if no other match

def clean_sheet(data, sheet_name):
    data = standardize_columns(data)
    data['Square Feet'] = data['Square Feet'].apply(clean_square_feet)
    data['Year Opened'] = data['Year Opened'].apply(robust_extract_year)
    data['Type'] = data['Description of Operation'].apply(lambda x: extract_type(x, sheet_name))
    return data[['State', 'DC Number', 'Location', 'Square Feet', 'Year Opened', 'Type']]

# Process and aggregate all sheets with the updated extraction functions
cleaned_data = pd.concat([clean_sheet(sheet, name) for name, sheet in all_sheets.items() if not sheet.empty])
summary_stats = cleaned_data.groupby(['Year Opened', 'Type']).agg({'Square Feet': 'sum', 'DC Number': 'count'}).reset_index()
summary_stats.rename(columns={'DC Number': 'Count'}, inplace=True)
summary_stats.sort_values(by='Year Opened', inplace=True)

# Output summary statistics
print(summary_stats)


In [ ]:
# Compute the cumulative totals for each year, segmented by type

# Filter out future years beyond the current year for realistic cumulative data
current_year = pd.to_datetime("now").year
cleaned_data_filtered = cleaned_data[cleaned_data['Year Opened'] <= current_year]

# Create a pivot table with the sum of square feet and count of DCs per year per type
pivot_data = pd.pivot_table(cleaned_data_filtered, values=['Square Feet', 'DC Number'], index='Type', 
                            columns='Year Opened', aggfunc={'Square Feet': 'sum', 'DC Number': 'count'}, fill_value=0)

# Cumulative totals across the years for square feet and count
cumulative_data = pivot_data.cumsum(axis=1)

# Prepare final table with types as rows and years as columns showing cumulative totals
# Adding total summary row at the bottom
cumulative_data.loc['Total', :] = cumulative_data.sum()
cumulative_data


In [1]:
# Here is the final clean code with all the latest changes for running in your own environment.

file_path = '/Users/myself/Desktop/Walmart USA Serching for Growth/Distribution Centers.xlsx'
final_code = """
import pandas as pd
import re

# Function to standardize the columns based on common variations encountered
def standardize_columns(data):
    standard_names = {
        'DC': 'DC Number',
        'Year': 'Year Opened',
        'Sq Ft': 'Square Feet',
        # Add other variations as needed
    }
    return data.rename(columns={old: standard_names.get(old, old) for old in data.columns})

# Function to clean the 'Square Feet' column
def clean_square_feet(value):
    value = str(value).replace(',', '').replace(' ', '')
    parts = value.split('.')
    if len(parts) > 2:
        value = ''.join(parts[:-1]) + '.' + parts[-1]
    try:
        return float(value)
    except ValueError:
        return 0.0

# Function to robustly extract the year from various formats, including handling merged cells or multiple rows
def final_robust_extract_year(date_str):
    if pd.isna(date_str):
        return None
    # Handling various common formats including quarters concatenated with years like Q32023
    year_match = re.search(r'(Q[1-4]\\s*,?\\s*)?(20|19)\\d{2}', str(date_str), re.IGNORECASE)
    if year_match:
        return int(year_match.group(0)[-4:])
    # Handle cases where 'Year Opened' is a textual description implying recent opening, such as "Opened in"
    if 'opened' in str(date_str).lower() or 'expected' in str(date_str).lower():
        # Assume the current year for recently opened entries
        return pd.to_datetime('now').year
    return None

# Function to extract the 'Type' from the 'Description of Operation' or use the sheet name
def extract_type(description, sheet_name):
    description = str(description)  # Convert to string to handle NaN values safely
    if 'Crossdock' in description:
        return 'Crossdock'
    elif 'Import' in description:
        return 'Import'
    elif 'Distribution Center' in description or 'General' in description:
        return 'Distribution Center'
    return sheet_name  # Use the sheet name as the type if no other match

# Function to clean each sheet
def clean_sheet(data, sheet_name):
    data = standardize_columns(data)
    data['Square Feet'] = data['Square Feet'].apply(clean_square_feet)
    data['Year Opened'] = data['Year Opened'].apply(final_robust_extract_year)
    data['Type'] = data['Description of Operation'].apply(lambda x: extract_type(x, sheet_name))
    return data[['State', 'DC Number', 'Location', 'Square Feet', 'Year Opened', 'Type']]

# Load the data from the Excel file
file_path = 'path_to_your_excel_file.xlsx'
all_sheets = pd.read_excel(file_path, sheet_name=None)

# Concatenate the cleaned data from all sheets
cleaned_data = pd.concat([clean_sheet(sheet, name) for name, sheet in all_sheets.items() if not sheet.empty], ignore_index=True)

# Display a sample of the cleaned data
print(cleaned_data.head())
"""

# Execute the final clean code in this environment to make sure it works
exec(final_code.replace("path_to_your_excel_file.xlsx", file_path))


        State DC Number                      Location  Square Feet  \
0         NaN    Number                           NaN          NaN   
1  California      8397    2063 Miguel Bustante Pkwy,     125800.0   
2         NaN       NaN     Colton, California, 92324          NaN   
3  California      6912  600 Live Oak Ave, Irwindale,     108850.0   
4         NaN       NaN             California, 91706          NaN   

   Year Opened          Type  
0       2024.0  Center Point  
1       2006.0     Crossdock  
2          NaN  Center Point  
3       2009.0     Crossdock  
4          NaN  Center Point  


In [2]:
# # Calculate the cumulative totals for each year for summary statistics
# # First, create a DataFrame that will contain the summary statistics with types as rows and years as columns
# unique_years = sorted(cleaned_data['Year Opened'].dropna().unique())
# summary_stats_df = pd.DataFrame(index=cleaned_data['Type'].unique(), columns=unique_years)

# # Initialize the DataFrame with zeros
# summary_stats_df.fillna(0, inplace=True)

# # Populate the DataFrame with cumulative counts and square footage per year
# for year in unique_years:
#     year_data = cleaned_data[cleaned_data['Year Opened'] <= year]
#     summary_stats_df[year] = year_data.groupby('Type').agg({'DC Number': 'count', 'Square Feet': 'sum'})

# # Calculate cumulative totals
# cumulative_summary_stats_df = summary_stats_df.cumsum(axis=1)

# # Adding a total summary row at the bottom that sums up the totals across all types for each year
# cumulative_summary_stats_df.loc['Total'] = cumulative_summary_stats_df.sum()

# cumulative_summary_stats_df.transpose()


In [3]:
# # Define a new DataFrame to hold the cumulative summary stats
# cumulative_summary_stats = pd.DataFrame(index=cleaned_data['Type'].unique())

# # Loop through each year, calculate the counts and square footage, and add them to the summary DataFrame
# for year in unique_years:
#     # Filter data for the current year or before
#     data_filtered_by_year = cleaned_data[cleaned_data['Year Opened'] <= year]
#     # Group by 'Type' and get the count of 'DC Number' and the sum of 'Square Feet'
#     stats_by_year = data_filtered_by_year.groupby('Type').agg({'DC Number': 'size', 'Square Feet': 'sum'})
#     # Rename columns to represent the current year
#     stats_by_year = stats_by_year.rename(columns={'DC Number': f'Count {year}', 'Square Feet': f'Square Feet {year}'})
#     # If the type isn't present in a given year, it should have a count and square feet of 0, not NaN after grouping
#     stats_by_year = stats_by_year.reindex(cumulative_summary_stats.index, fill_value=0)
#     # Add the current year's stats to the cumulative summary stats
#     cumulative_summary_stats = pd.concat([cumulative_summary_stats, stats_by_year], axis=1)

# # Replace NaNs with 0 for cumulative calculations
# cumulative_summary_stats.fillna(0, inplace=True)

# # Calculate cumulative totals
# for col in cumulative_summary_stats.columns:
#     if 'Count' in col:  # Cumulative count for each type of center
#         cumulative_summary_stats[col] = cumulative_summary_stats.filter(regex='Count').cumsum(axis=1)[col]
#     elif 'Square Feet' in col:  # Cumulative square footage for each type of center
#         cumulative_summary_stats[col] = cumulative_summary_stats.filter(regex='Square Feet').cumsum(axis=1)[col]

# # Add a row for the total summary
# total_summary = cumulative_summary_stats.sum(axis=0)
# total_summary.name = 'Total'
# cumulative_summary_stats = cumulative_summary_stats.append(total_summary)

# # Transpose the DataFrame for better readability
# cumulative_summary_stats = cumulative_summary_stats.transpose()

# # Separate out the count and square footage into two DataFrames for clearer presentation
# counts_df = cumulative_summary_stats.filter(regex='Count').copy()
# square_feet_df = cumulative_summary_stats.filter(regex='Square Feet').copy()

# # Show cumulative counts and square footage per year
# counts_df, square_feet_df


In [4]:
 # Create a pivot table for the counts of distribution centers
pivot_counts = cleaned_data.pivot_table(
    values='DC Number', index='Type', columns='Year Opened', aggfunc='count', fill_value=0)

# Create a pivot table for the square footage
pivot_square_feet = cleaned_data.pivot_table(
    values='Square Feet', index='Type', columns='Year Opened', aggfunc='sum', fill_value=0)

# Calculate the cumulative totals across the years for both counts and square footage
cumulative_counts = pivot_counts.cumsum(axis=1)
cumulative_square_feet = pivot_square_feet.cumsum(axis=1)

# Append total summary rows for counts and square footage
cumulative_counts.loc['Total'] = cumulative_counts.sum(axis=0)
cumulative_square_feet.loc['Total'] = cumulative_square_feet.sum(axis=0)

# Combine both cumulative dataframes for a full summary
cumulative_summary = pd.concat([cumulative_counts, cumulative_square_feet], keys=['Counts', 'Square Feet'])

# Transpose for the desired layout
cumulative_summary = cumulative_summary.transpose()

# Ensure we have the years as columns from earliest to latest
cumulative_summary = cumulative_summary.reindex(sorted(cumulative_summary.columns), axis=1)

cumulative_summary.tail()  # Show the last few rows to see the most recent years


Counts                                                      \
Type        Center Point Crossdock Distribution Center Food Food Manufacture   
Year Opened                                                                    
2023.0               1.0      30.0                79.0  0.0              0.0   
2024.0               2.0      31.0                79.0  0.0              1.0   
2025.0               2.0      31.0                79.0  0.0              1.0   
2026.0               2.0      31.0                79.0  0.0              1.0   
2027.0               2.0      31.0                79.0  0.0              1.0   

                                                                              \
Type        Footwear Import Regional General Merchandise Sams eCommerce Dark   
Year Opened                                                                    
2023.0           2.0   15.0                         34.0                 4.0   
2024.0           3.0   16.0                         34.0                 5.0   
2025.0           3.0   16.0                         34.0                 5.0   
2026.0           3.0   16.0                         34.0                 5.0   
2027.0           3.0   16.0                         34.0                 5.0   

                       ... Square Feet                              \
Type        Specialty  ...        Food Food Manufacture   Footwear   
Year Opened            ...                                           
2023.0            0.0  ...         0.0         201000.0  1630860.0   
2024.0            0.0  ...         0.0         201000.0  1630860.0   
2025.0            0.0  ...         0.0         531671.0  1630860.0   
2026.0            0.0  ...         0.0         531671.0  1630860.0   
2027.0            0.0  ...         0.0         531671.0  1630860.0   

                                                                          \
Type             Import Regional General Merchandise Sams eCommerce Dark   
Year Opened                                                                
2023.0       11571025.0                   41255500.0            555460.8   
2024.0       11571025.0                   41255500.0            555460.8   
2025.0       11571025.0                   41255500.0            555460.8   
2026.0       11571025.0                   41255500.0            555460.8   
2027.0       11571025.0                   41255500.0            555460.8   

                                                              
Type        Specialty         Total    Vacated     eCommerce  
Year Opened                                                   
2023.0            0.0  1.304057e+08  4624134.0  1.094417e+07  
2024.0       365000.0  1.353894e+08  4624134.0  1.204417e+07  
2025.0       365000.0  1.357201e+08  4624134.0  1.204417e+07  
2026.0       365000.0  1.366201e+08  4624134.0  1.294417e+07  
2027.0       365000.0  1.378201e+08  4624134.0  1.294417e+07  

[5 rows x 26 columns]

In [5]:
cumulative_summary['Counts']

Type,Center Point,Crossdock,Distribution Center,Food,Food Manufacture,Footwear,Import,Regional General Merchandise,Sams eCommerce Dark,Specialty,Total,Vacated,eCommerce
Year Opened,,,,,,,,,,,,,
1970.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1975.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1976.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
1978.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
1980.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0
1981.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0
1983.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,1.0,0.0
1985.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,7.0,1.0,0.0
1986.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,10.0,2.0,0.0


In [6]:
# Save the cumulative summary to an Excel file
output_file_path = '/Users/myself/Desktop/Walmart USA Serching for Growth/Cumulative_Summary_Distribution_Centers.xlsx'
cumulative_summary.to_excel(output_file_path, sheet_name='Cumulative Summary')

print(f'Cumulative summary saved to {output_file_path}')


Cumulative summary saved to /Users/myself/Desktop/Walmart USA Serching for Growth/Cumulative_Summary_Distribution_Centers.xlsx
